In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install -q pytorch-lightning==2.4.0 lightning==2.4.0 pytorch-forecasting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import torch
from datetime import datetime, timedelta

from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet, QuantileLoss
from pytorch_forecasting.data import GroupNormalizer
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
import lightning.pytorch as pl

torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

import sys
import torch

print("Python exe:", sys.executable)
print("PyTorch version:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())


PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA device: Tesla T4
Python exe: /usr/bin/python3
PyTorch version: 2.8.0+cu126
torch.version.cuda: 12.6
CUDA available: True


In [ ]:
!pip install pandas_ta

In [ ]:

def get_forex_data_enhanced(symbol='GBPUSD=X', start_date='2025-9-23', end_date='2025-11-19',
                           interval='5m', extend_days=10):



    data = yf.download(symbol, start=start_date, end=end_date,
                      interval=interval, progress=True)

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]

    data.index = pd.to_datetime(data.index, utc=True)
    data.index.name = 'timestamp'

    print(f"Obtenidos {len(data)} registros")

    return data

def calculate_log_returns_multi_horizon(data, periods=[1, 3, 6, 12, 24, 48]):
    df = data.copy()

    for period in periods:
        df[f'log_return_{period}p'] = np.log(df['Close'] / df['Close'].shift(period))

        df[f'future_log_return_{period}p'] = df[f'log_return_{period}p'].shift(-period)

        time_minutes = period * 5
        valid_count = df[f'log_return_{period}p'].count()
        print(f"Periodo {period} ({time_minutes}min): {valid_count} valores válidos")

    df['target_log_return'] = df['future_log_return_1p']

    if 'target_log_return' in df.columns:
        print(f"\nEstadísticas Target Log Return:")
        print(f"  Media: {df['target_log_return'].mean():.6f}")
        print(f"  Std: {df['target_log_return'].std():.6f}")
        print(f"  Valores válidos: {df['target_log_return'].count()} / {len(df)}")

    return df

def add_technical_indicators_comprehensive(data):
    df = data.copy()

    def calculate_rsi(prices, period=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    df['rsi_14'] = calculate_rsi(df['Close'], 14)
    df['rsi_7'] = calculate_rsi(df['Close'], 7)
    df['sma_10'] = df['Close'].rolling(10).mean()
    df['sma_20'] = df['Close'].rolling(20).mean()
    df['sma_50'] = df['Close'].rolling(50).mean()
    df['ema_12'] = df['Close'].ewm(span=12).mean()
    df['ema_26'] = df['Close'].ewm(span=26).mean()
    df['macd'] = df['ema_12'] - df['ema_26']
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_histogram'] = df['macd'] - df['macd_signal']
    df['bb_middle'] = df['sma_20']
    bb_std = df['Close'].rolling(20).std()
    df['bb_upper'] = df['bb_middle'] + (bb_std * 2)
    df['bb_lower'] = df['bb_middle'] - (bb_std * 2)
    df['bb_position'] = (df['Close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr_14'] = true_range.rolling(14).mean()
    for period in [5, 10, 20, 50]:
        df[f'price_momentum_{period}'] = df['Close'] / df['Close'].shift(period) - 1
    for period in [10, 20, 50]:
        df[f'volatility_{period}'] = df['log_return_1p'].rolling(period).std()

    # High/Low ratios
    df['high_low_ratio'] = df['High'] / df['Low']
    df['close_position'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])

    print(f"Indicadores técnicos calculados")
    return df

def add_temporal_features_forex(data):
    df = data.copy()
    print("Añadiendo features temporales forex...")

    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['minute'] = df.index.minute
    df['tokyo_session'] = ((df['hour'] >= 23) | (df['hour'] < 8)).astype(int)
    df['london_session'] = ((df['hour'] >= 8) & (df['hour'] < 17)).astype(int)
    df['new_york_session'] = ((df['hour'] >= 13) & (df['hour'] < 22)).astype(int)
    df['london_ny_overlap'] = (df['london_session'] & df['new_york_session']).astype(int)
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_high_activity'] = (
        (df['london_ny_overlap'] == 1) |
        ((df['hour'] >= 8) & (df['hour'] <= 10)) |
        ((df['hour'] >= 13) & (df['hour'] <= 15))
    ).astype(int)

    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

    print(f"Features temporales añadidos")
    return df

print("\n" + "=" * 60)
print("PIPELINE DE FEATURES")
print("=" * 60)

data = get_forex_data_enhanced()
datos_with_returns = calculate_log_returns_multi_horizon(data)
datos_with_tech = add_technical_indicators_comprehensive(datos_with_returns)
datos_with_temporal = add_temporal_features_forex(datos_with_tech)

print(f"\n eatures pipeline completado:")
print(f"  Registros: {len(datos_with_temporal)}")
print(f"  Columnas: {len(datos_with_temporal.columns)}")

# **IMPLEMENTACIÓN (parcial) DE TFT**

In [ ]:
def clean_data_for_tft_robust(data):
    print("Limpieza robusta de datos para TFT...")

    df = data.copy()
    original_len = len(df)
    print(f"Datos originales: {original_len} registros")

    target_col = 'target_log_return'

    if target_col in df.columns:
        print(f"\nLimpiando {target_col}...")

        total_vals = len(df[target_col])
        nan_vals = df[target_col].isna().sum()
        inf_vals = np.isinf(df[target_col]).sum()

        print(f"Antes - Total: {total_vals}, NaN: {nan_vals}, Inf: {inf_vals}")
        print(f"Porcentaje problemático: {(nan_vals + inf_vals)/total_vals*100:.2f}%")

        df = df[~np.isinf(df[target_col])]
        df = df[~df[target_col].isna()]
        print(f"Después de limpiar target: {len(df)} registros")

        if len(df) > 0:
            mean_ret = df[target_col].mean()
            std_ret = df[target_col].std()

            if std_ret > 0:
                outlier_threshold = 8  
                outlier_filter = np.abs((df[target_col] - mean_ret) / std_ret) <= outlier_threshold
                df = df[outlier_filter]
                print(f"Después de remover outliers extremos: {len(df)} registros")

    price_vars = ['Close', 'High', 'Low', 'Open']
    for var in price_vars:
        if var in df.columns:
            df = df[df[var] > 0]
            df = df[~np.isinf(df[var])]
            df = df[~df[var].isna()]

    print(f"Después de limpiar precios: {len(df)} registros")

    technical_indicators = ['rsi_14', 'macd', 'bb_position', 'atr_14']

    for indicator in technical_indicators:
        if indicator in df.columns:
            df = df[~np.isinf(df[indicator])]
            df[indicator] = df[indicator].fillna(method='ffill', limit=5)
            df = df[~df[indicator].isna()]

    print(f"Después de limpiar indicadores: {len(df)} registros")

    # 4. VERIFICACIÓN FINAL
    remaining_len = len(df)
    reduction_pct = (original_len - remaining_len) / original_len * 100

    print(f"\nRESUMEN DE LIMPIEZA:")
    print(f"  Datos originales: {original_len}")
    print(f"  Datos finales: {remaining_len}")
    print(f"  Reducción: {reduction_pct:.1f}%")

    if remaining_len < 1000:
        print(f"Warning: Pocos datos restantes ({remaining_len})")

    # 5. VERIFICAR target_log_return FINAL
    if target_col in df.columns and len(df) > 0:
        final_nan = df[target_col].isna().sum()
        final_inf = np.isinf(df[target_col]).sum()

        print(f"\nVERIFICACIÓN FINAL {target_col}:")
        print(f"  NaN finales: {final_nan}")
        print(f"  Infinitos finales: {final_inf}")

        if final_nan > 0 or final_inf > 0:
            print("Aún hay valores problemáticos!")
            return None
        else:
            print("target_log_return completamente limpio!")

    return df

def prepare_data_for_tft_final(data, start_date='2025-9-23'):
    print("Preparación final para TFT...")

    clean_data = clean_data_for_tft_robust(data)

    if clean_data is None or len(clean_data) < 500:
        raise ValueError(f"Datos insuficientes después de limpieza: {len(clean_data) if clean_data is not None else 0}")

    df = clean_data.copy()
    df = df.reset_index()
    df['time_idx'] = range(len(df))
    df['group_id'] = 'GBPUSD'
    start_date_utc = pd.to_datetime(start_date).tz_localize('UTC')
    df = df[df['timestamp'] >= start_date_utc]
    df['time_idx'] = range(len(df))  

    print(f" Datos finales: {len(df)} registros")
    print(f" Rango: {df['timestamp'].min()} - {df['timestamp'].max()}")

    return df

print("\n" + "=" * 60)
print("LIMPIEZA DE DATOS")
print("=" * 60)

datos_final = prepare_data_for_tft_final(datos_with_temporal)

print(f"\nDatos listos para TFT:")
print(f"  Shape: {datos_final.shape}")
print(f"  Período: {(datos_final['timestamp'].max() - datos_final['timestamp'].min()).days} días")
print(datos_final.head())

In [ ]:
def create_working_tft_datasets(train_data, val_data, test_data):

    print("Creando datasets TFT con configuración válida...")

    time_varying_known_reals = [
        "hour", "day_of_week", "minute",
        "tokyo_session", "london_session", "new_york_session",
        "london_ny_overlap", "is_weekend", "is_high_activity",
        "hour_sin", "hour_cos", "dow_sin", "dow_cos"
    ]

    time_varying_unknown_reals = [
        "target_log_return",
        "log_return_1p", "log_return_3p", "log_return_6p",
        "Close", "High", "Low", "Open",
        "rsi_14", "rsi_7",
        "macd", "macd_signal", "macd_histogram",
        "bb_position", "bb_width",
        "atr_14",
        "price_momentum_5", "price_momentum_10",
        "volatility_10"
    ]

    time_varying_known_reals = [col for col in time_varying_known_reals if col in train_data.columns]
    time_varying_unknown_reals = [col for col in time_varying_unknown_reals if col in train_data.columns]

    print(f"  Variables conocidas: {len(time_varying_known_reals)}")
    print(f"  Variables no conocidas: {len(time_varying_unknown_reals)}")

    for name, data in [("Train", train_data), ("Val", val_data), ("Test", test_data)]:
        target_nans = data['target_log_return'].isna().sum()
        target_infs = np.isinf(data['target_log_return']).sum()
        print(f"  {name}: {len(data)} registros, target clean ({target_nans} NaN, {target_infs} inf)")

    try:
        print("\n Probando configuración estándar...")

        training_data = TimeSeriesDataSet(
            train_data,
            time_idx="time_idx",
            target="target_log_return",
            group_ids=["group_id"],

            min_encoder_length=30,
            max_encoder_length=60,
            min_prediction_length=1,
            max_prediction_length=12,

            time_varying_known_reals=time_varying_known_reals,
            time_varying_unknown_reals=time_varying_unknown_reals,

            target_normalizer=GroupNormalizer(
                groups=["group_id"],
                transformation="softplus"  
            ),

            allow_missing_timesteps=True,
            add_relative_time_idx=True,
            add_target_scales=True,
            add_encoder_length=True,
        )

        print("  Training dataset creado!")

        validation_data = TimeSeriesDataSet.from_dataset(
            training_data, val_data, predict=True, stop_randomization=True
        )

        test_dataset = TimeSeriesDataSet.from_dataset(
            training_data, test_data, predict=True, stop_randomization=True
        )

        print("  Todos los datasets creados exitosamente!")
        return training_data, validation_data, test_dataset

    except Exception as e:
        print(f"Error con configuración estándar: {e}")

        print("\nUsando configuración mínima garantizada...")

        minimal_known = ["hour", "day_of_week", "minute"]
        minimal_unknown = ["target_log_return", "Close", "log_return_1p"]

        minimal_known = [col for col in minimal_known if col in train_data.columns]
        minimal_unknown = [col for col in minimal_unknown if col in train_data.columns]

        print(f" Variables mínimas - conocidas: {len(minimal_known)}, no conocidas: {len(minimal_unknown)}")

        training_minimal = TimeSeriesDataSet(
            train_data,
            time_idx="time_idx",
            target="target_log_return",
            group_ids=["group_id"],

            min_encoder_length=20,
            max_encoder_length=40,
            min_prediction_length=1,
            max_prediction_length=6,

            time_varying_known_reals=minimal_known,
            time_varying_unknown_reals=minimal_unknown,
            allow_missing_timesteps=True,
            add_relative_time_idx=True,
        )

        validation_minimal = TimeSeriesDataSet.from_dataset(
            training_minimal, val_data, predict=True, stop_randomization=True
        )

        test_minimal = TimeSeriesDataSet.from_dataset(
            training_minimal, test_data, predict=True, stop_randomization=True
        )

        print("  Datasets mínimos creados!")
        return training_minimal, validation_minimal, test_minimal

print("CREANDO DATASETS TFT CON CONFIGURACIÓN VÁLIDA")
print("=" * 60)

training_data, validation_data, test_dataset = create_working_tft_datasets(
    train_clean, val_clean, test_clean
)

print(f"\n Creando modelo TFT optimizado...")

tft_model = TemporalFusionTransformer.from_dataset(
    training_data,
    loss=QuantileLoss(quantiles=[0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]),

    hidden_size=32,
    attention_head_size=2,
    dropout=0.2,
    hidden_continuous_size=4,

    learning_rate=0.01,
    log_interval=50,
    reduce_on_plateau_patience=5,
)

total_params = sum(p.numel() for p in tft_model.parameters())
print(f"  Modelo TFT creado: {total_params:,} parámetros (~{total_params/1e6:.1f}M)")

train_dataloader = training_data.to_dataloader(train=True, batch_size=32, num_workers=0)
val_dataloader = validation_data.to_dataloader(train=False, batch_size=32, num_workers=0)
test_dataloader = test_dataset.to_dataloader(train=False, batch_size=32, num_workers=0)

train_raw = train_clean
val_raw = val_clean
test_raw = test_clean

print(f"\n¡DATASETS Y MODELO CREADOS EXITOSAMENTE!")
print(f"  Train batches: {len(train_dataloader)}")
print(f"  Val batches: {len(val_dataloader)}")
print(f"  Test batches: {len(test_dataloader)}")
print(f"  Cuantiles: [2%, 10%, 25%, 50%, 75%, 90%, 98%]")

print(f"\n✅ LISTO PARA ENTRENAMIENTO!")
print(f"💡 Usa: trained_model, trainer = train_tft_model_robust(tft_model, train_dataloader, val_dataloader)")

In [ ]:
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl

def train_tft_model_robust(model, train_dataloader, val_dataloader, max_epochs=25):

    print("Iniciando entrenamiento TFT robusto...")

    train_batches = len(train_dataloader)

    if train_batches < 50:
        patience = 5
        max_epochs = min(max_epochs, 20)
        print(f"  Dataset pequeño - ajustando config")
    else:
        patience = 8
        print(f"  Dataset normal")

    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        patience=patience,
        verbose=True,
        mode="min",
        min_delta=0.0001
    )

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1,
        enable_model_summary=True,
        gradient_clip_val=0.1,
        callbacks=[early_stop_callback],
        log_every_n_steps=min(50, max(10, train_batches // 4)),
        enable_progress_bar=True,
    )

    print(f"  Max epochs: {max_epochs}")
    print(f"  Early stopping patience: {patience}")
    print(f"  Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

    try:
        print("\nIniciando entrenamiento...")

        trainer.fit(
            model,
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader,
        )

        print("\nEntrenamiento completado exitosamente!")
        return model, trainer

    except Exception as e:
        print(f"\nError durante entrenamiento: {e}")
        print(" Intentando con configuración conservadora...")

        early_stop_simple = EarlyStopping(monitor="val_loss", patience=3, mode="min")

        trainer_conservative = pl.Trainer(
            max_epochs=min(max_epochs, 10),
            accelerator="cpu",
            devices=1,
            enable_model_summary=False,
            gradient_clip_val=0.05,
            callbacks=[early_stop_simple],
            enable_progress_bar=False,
        )

        trainer_conservative.fit(
            model,
            train_dataloaders=train_dataloader,
            val_dataloaders=val_dataloader,
        )

        print("Entrenamiento conservador completado!")
        return model, trainer_conservative

print("INICIANDO ENTRENAMIENTO DEL TFT")
print("=" * 60)

trained_model, trainer = train_tft_model_robust(
    tft_model,
    train_dataloader,
    val_dataloader,
    max_epochs=25
)

print(f"\n ¡MODELO ENTRENADO EXITOSAMENTE!")

In [ ]:
def generate_predictions_with_quantiles_safe(model, dataloader, dataset_name="Validation"):

    print(f"Generando predicciones en {dataset_name} set...")

    try:
        model.eval()  
        raw_predictions = model.predict(dataloader, mode="raw", return_x=True)
        predictions = raw_predictions.output['prediction']

        if torch.is_tensor(predictions):
            predictions_np = predictions.cpu().numpy()
        else:
            predictions_np = predictions

        batch_size, time_horizons, num_quantiles = predictions_np.shape
        quantiles = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]

        print(f"  📊 Predicciones shape: {predictions_np.shape}")
        print(f"  📊 Batches: {batch_size}, Horizontes: {time_horizons}, Cuantiles: {num_quantiles}")

        formatted_predictions = {
            'quantiles': quantiles,
            'predictions': predictions_np,

            'confidence_intervals': {
                '95%': {
                    'lower': predictions_np[:, :, 0],  
                    'upper': predictions_np[:, :, 6],  
                },
                '80%': {
                    'lower': predictions_np[:, :, 1],  
                    'upper': predictions_np[:, :, 5],
                },
                '50%': {
                    'lower': predictions_np[:, :, 2], 
                    'upper': predictions_np[:, :, 4], 
                }
            },

            'median': predictions_np[:, :, 3],  
            'uncertainty': predictions_np[:, :, 6] - predictions_np[:, :, 0], 
            'iqr': predictions_np[:, :, 4] - predictions_np[:, :, 2],  
            'skew_proxy': (predictions_np[:, :, 3] - predictions_np[:, :, 0]) / (predictions_np[:, :, 6] - predictions_np[:, :, 0] + 1e-8),

           
            'var_10': predictions_np[:, :, 1],  
            'var_5': predictions_np[:, :, 0],   
        }

        print(f"\nEstadísticas de predicciones:")
        print(f"  Mediana promedio: {formatted_predictions['median'].mean():.6f}")
        print(f"  Mediana std: {formatted_predictions['median'].std():.6f}")
        print(f"  Incertidumbre promedio: {formatted_predictions['uncertainty'].mean():.6f}")
        print(f"  IQR promedio: {formatted_predictions['iqr'].mean():.6f}")

        return formatted_predictions, raw_predictions

    except Exception as e:
        print(f"Error generando predicciones: {e}")
        return None, None

print("\n GENERANDO PREDICCIONES EN VALIDATION SET")
print("=" * 60)

predictions, raw_output = generate_predictions_with_quantiles_safe(
    trained_model,
    val_dataloader,
    "Validation"
)

if predictions is not None:
    print("Predicciones en validation generadas exitosamente!")
else:
    print("Error en predicciones de validation")

In [ ]:
def evaluate_on_test_set_comprehensive(model, test_dataloader, test_raw_data):

    print("Evaluando en test set...")

    try:
        model.eval()
        test_predictions = model.predict(test_dataloader, mode="raw", return_x=True)
        predictions_test = test_predictions.output['prediction']

        if torch.is_tensor(predictions_test):
            predictions_np = predictions_test.cpu().numpy()
        else:
            predictions_np = predictions_test

        median_predictions = predictions_np[:, :, 3]  
        uncertainty = predictions_np[:, :, 6] - predictions_np[:, :, 0]  
        var_10 = predictions_np[:, :, 1]  
        var_5 = predictions_np[:, :, 0]   
        test_performance = {
            'predictions_shape': predictions_np.shape,
            'samples_count': len(test_raw_data),
            'prediction_horizon_periods': predictions_np.shape[1],
            'prediction_horizon_minutes': f"{predictions_np.shape[1]*5}min",
            'median_mean': median_predictions.mean(),
            'median_std': median_predictions.std(),
            'median_min': median_predictions.min(),
            'median_max': median_predictions.max(),
            'uncertainty_mean': uncertainty.mean(),
            'uncertainty_std': uncertainty.std(),
            'var_10_mean': var_10.mean(),
            'var_5_mean': var_5.mean(),
            'test_period': f"{test_raw_data['timestamp'].min()} → {test_raw_data['timestamp'].max()}",
            'test_days': (test_raw_data['timestamp'].max() - test_raw_data['timestamp'].min()).days,
        }

        print(f"\nPERFORMANCE DETALLADO EN TEST SET:")
        print(f" Shape predicciones: {test_performance['predictions_shape']}")
        print(f" Samples test: {test_performance['samples_count']}")
        print(f" Horizonte: {test_performance['prediction_horizon_periods']} períodos ({test_performance['prediction_horizon_minutes']})")
        print(f" Período test: {test_performance['test_days']} días")

        print(f"\nEstadísticas predicciones:")
        print(f"  Mediana promedio: {test_performance['median_mean']:.6f}")
        print(f"  Mediana std: {test_performance['median_std']:.6f}")
        print(f"  Mediana rango: {test_performance['median_min']:.6f} → {test_performance['median_max']:.6f}")

        print(f"\nRisk metrics:")
        print(f"  Incertidumbre promedio: {test_performance['uncertainty_mean']:.6f}")
        print(f"  VaR 10%: {test_performance['var_10_mean']:.6f}")
        print(f"  VaR 5%: {test_performance['var_5_mean']:.6f}")

        return predictions_np, test_performance

    except Exception as e:
        print(f"Error en evaluación de test: {e}")
        return None, None

print("\n EVALUACIÓN FINAL EN TEST SET")
print("=" * 60)

test_predictions_np, final_performance = evaluate_on_test_set_comprehensive(
    trained_model,
    test_dataloader,
    test_raw
)

if test_predictions_np is not None:
    print("\nEvaluación en test completada exitosamente!")
else:
    print("Error en evaluación de test")

In [ ]:
def format_for_meta_model_comprehensive(predictions, model_name='TFT'):
   
    print(f"Formateando predicciones {model_name} para meta-modelo...")

    if predictions is None:
        print("No hay predicciones para formatear")
        return None

    meta_features = {
        f'{model_name}_median': predictions['median'],
        f'{model_name}_q02': predictions['predictions'][:, :, 0],  
        f'{model_name}_q10': predictions['predictions'][:, :, 1],  
        f'{model_name}_q25': predictions['predictions'][:, :, 2],  
        f'{model_name}_q75': predictions['predictions'][:, :, 4],  
        f'{model_name}_q90': predictions['predictions'][:, :, 5],  
        f'{model_name}_q98': predictions['predictions'][:, :, 6],  

        f'{model_name}_uncertainty': predictions['uncertainty'],
        f'{model_name}_iqr': predictions['iqr'],
        f'{model_name}_skew': predictions['skew_proxy'],

        f'{model_name}_prob_up': (predictions['median'] > 0).astype(float),
        f'{model_name}_prob_down': (predictions['median'] < 0).astype(float),
        f'{model_name}_strong_up': (predictions['predictions'][:, :, 5] > 0.001).astype(float), 
        f'{model_name}_strong_down': (predictions['predictions'][:, :, 1] < -0.001).astype(float),  
        f'{model_name}_neutral': ((predictions['median'] >= -0.0005) & (predictions['median'] <= 0.0005)).astype(float),

        f'{model_name}_var_10': predictions['var_10'],
        f'{model_name}_var_5': predictions['var_5'],
        f'{model_name}_cvar_10': predictions['predictions'][:, :, 1],  
        f'{model_name}_cvar_5': predictions['predictions'][:, :, 0],   

        f'{model_name}_confidence_95': 1 / (1 + predictions['confidence_intervals']['95%']['upper'] - predictions['confidence_intervals']['95%']['lower']),
        f'{model_name}_confidence_80': 1 / (1 + predictions['confidence_intervals']['80%']['upper'] - predictions['confidence_intervals']['80%']['lower']),
        f'{model_name}_confidence_50': 1 / (1 + predictions['confidence_intervals']['50%']['upper'] - predictions['confidence_intervals']['50%']['lower']),

        f'{model_name}_expected_vol': predictions['uncertainty'] / 4,  
        f'{model_name}_upside_vol': predictions['predictions'][:, :, 5] - predictions['median'],  
        f'{model_name}_downside_vol': predictions['median'] - predictions['predictions'][:, :, 1],  

        f'{model_name}_bullish_momentum': np.where(predictions['median'] > 0, predictions['median'] * predictions['confidence_intervals']['80%']['upper'], 0),
        f'{model_name}_bearish_momentum': np.where(predictions['median'] < 0, predictions['median'] * predictions['confidence_intervals']['80%']['lower'], 0),
    }

    print(f"  {len(meta_features)} features creados para meta-modelo")

    print(f"  Estadísticas de features:")
    print(f"    Shape: {meta_features[f'{model_name}_median'].shape}")
    print(f"    Horizontes: 1-{meta_features[f'{model_name}_median'].shape[1]} períodos")
    print(f"    Samples: {meta_features[f'{model_name}_median'].shape[0]}")

    feature_categories = {
        'quantiles': 7,
        'uncertainty': 3,
        'directional': 5,
        'risk_management': 4,
        'confidence': 3,
        'volatility': 3,
        'momentum': 2
    }

    print(f" Categorías de features:")
    for category, count in feature_categories.items():
        print(f"    {category}: {count} features")

    return meta_features

if predictions is not None:
    print("\n PREPARANDO FEATURES PARA META-MODELO")
    print("=" * 60)

    meta_model_features = format_for_meta_model_comprehensive(predictions, 'TFT')

    if meta_model_features is not None:
        print("Features para meta-modelo preparados!")
    else:
        print("Error preparando features para meta-modelo")

In [ ]:
def plot_tft_results_fixed(predictions, test_performance=None, num_samples=20):

    if predictions is None:
        print("No hay predicciones para visualizar")
        return

    print("Creando visualizaciones corregidas...")

    try:
        import matplotlib.pyplot as plt
        import numpy as np

        pred_shape = predictions['predictions'].shape
        print(f"   Debug: Predictions shape = {pred_shape}")
        print(f"   Debug: Num samples requested = {num_samples}")
        print(f"   Debug: Median range = {predictions['median'].min():.8f} to {predictions['median'].max():.8f}")

        actual_samples = min(num_samples, pred_shape[0])
        max_horizons = pred_shape[1]

        print(f" Usando {actual_samples} samples, {max_horizons} horizontes")

        fig = plt.figure(figsize=(20, 12))
        ax1 = plt.subplot(2, 3, 1)

        if max_horizons >= 4:
            horizons_to_plot = [0, max_horizons//4, max_horizons//2, max_horizons-1]
        else:
            horizons_to_plot = list(range(max_horizons))

        colors = ['blue', 'green', 'orange', 'red', 'purple']

        for i, horizon in enumerate(horizons_to_plot):
            if horizon < max_horizons:
                color = colors[i % len(colors)]
                x_samples = list(range(actual_samples))

                median_data = predictions['median'][:actual_samples, horizon]
                lower_95 = predictions['confidence_intervals']['95%']['lower'][:actual_samples, horizon]
                upper_95 = predictions['confidence_intervals']['95%']['upper'][:actual_samples, horizon]

                print(f"    H{horizon}: median range = {median_data.min():.8f} to {median_data.max():.8f}")

                ax1.fill_between(x_samples, lower_95, upper_95,
                               alpha=0.2, color=color, label=f'95% CI H{horizon+1}')

                ax1.plot(x_samples, median_data, color=color, linewidth=2,
                        label=f'Median H{horizon+1}', marker='o', markersize=3)

        ax1.set_title('Predicciones Multi-Horizonte', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Muestra')
        ax1.set_ylabel('Log Return Predicho')
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax1.grid(True, alpha=0.3)

        ax2 = plt.subplot(2, 3, 2)
        uncertainty_flat = predictions['uncertainty'].flatten()
        uncertainty_flat = uncertainty_flat[~np.isnan(uncertainty_flat)] 

        if len(uncertainty_flat) > 0:
            ax2.hist(uncertainty_flat, bins=50, alpha=0.7, color='orange', edgecolor='black')
            ax2.set_title('Distribución de Incertidumbre')
            ax2.set_xlabel('Incertidumbre (q98% - q2%)')
            ax2.set_ylabel('Frecuencia')
        else:
            ax2.text(0.5, 0.5, 'No hay datos\nde incertidumbre', ha='center', va='center')
            ax2.set_title('Distribución de Incertidumbre - Sin Datos')
        ax2.grid(True, alpha=0.3)

        ax3 = plt.subplot(2, 3, 3)

        prob_up = (predictions['median'] > 0).astype(float)
        prob_down = (predictions['median'] < 0).astype(float)

        prob_up_avg = prob_up.mean(axis=1)[:actual_samples]
        prob_down_avg = prob_down.mean(axis=1)[:actual_samples]

        print(f"    Prob up range: {prob_up_avg.min():.3f} to {prob_up_avg.max():.3f}")
        print(f"    Prob down range: {prob_down_avg.min():.3f} to {prob_down_avg.max():.3f}")

        x_samples = list(range(actual_samples))
        ax3.plot(x_samples, prob_up_avg, 'g-', label='Prob. Alza', linewidth=2, marker='o', markersize=2)
        ax3.plot(x_samples, prob_down_avg, 'r-', label='Prob. Baja', linewidth=2, marker='s', markersize=2)
        ax3.axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, label='Neutral')

        ax3.set_title('Señales Direccionales')
        ax3.set_xlabel('Muestra')
        ax3.set_ylabel('Probabilidad Promedio')
        ax3.set_ylim(0, 1)
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        ax4 = plt.subplot(2, 3, 4)
        var_5_data = predictions['var_5'][:actual_samples, 0]
        var_10_data = predictions['var_10'][:actual_samples, 0]

        print(f"    VaR 5% range: {var_5_data.min():.8f} to {var_5_data.max():.8f}")
        print(f"    VaR 10% range: {var_10_data.min():.8f} to {var_10_data.max():.8f}")

        x_samples = list(range(actual_samples))
        ax4.plot(x_samples, var_5_data, 'r-', label='VaR 5%', linewidth=2, marker='v', markersize=3)
        ax4.plot(x_samples, var_10_data, 'orange', label='VaR 10%', linewidth=2, marker='^', markersize=3)
        ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5, label='Zero line')

        ax4.set_title('Value at Risk (Horizonte 1)')
        ax4.set_xlabel('Muestra')
        ax4.set_ylabel('VaR')
        ax4.legend()
        ax4.grid(True, alpha=0.3)

        ax5 = plt.subplot(2, 3, 5)

        sample_idx = actual_samples // 2
        horizon_idx = 0  

        quantiles = predictions['quantiles']
        quantile_values = predictions['predictions'][sample_idx, horizon_idx, :]

        print(f"    Quantile values range: {quantile_values.min():.8f} to {quantile_values.max():.8f}")

        ax5.plot(quantiles, quantile_values, 'bo-', linewidth=2, markersize=8)
        ax5.fill_between(quantiles, quantile_values, alpha=0.3, color='blue')
        ax5.set_title(f'Distribución de Cuantiles\n(Muestra {sample_idx+1}, H1)')
        ax5.set_xlabel('Cuantil')
        ax5.set_ylabel('Valor Predicho')
        ax5.grid(True, alpha=0.3)
        ax6 = plt.subplot(2, 3, 6)
        ax6.axis('off')

        performance_text = "RESUMEN TFT:\n\n"
        performance_text += f"PREDICCIONES:\n"
        performance_text += f"  Shape: {pred_shape}\n"
        performance_text += f"  Samples: {pred_shape[0]:,}\n"
        performance_text += f"  Horizontes: {pred_shape[1]}\n"
        performance_text += f"  Cuantiles: {pred_shape[2]}\n\n"

        performance_text += f"ESTADÍSTICAS:\n"
        performance_text += f"  Mediana: {predictions['median'].mean():.6f}\n"
        performance_text += f"  Std: {predictions['median'].std():.6f}\n"
        performance_text += f"  Incertidumbre: {predictions['uncertainty'].mean():.6f}\n"
        performance_text += f"  IQR: {predictions['iqr'].mean():.6f}\n\n"

        if test_performance:
            performance_text += f"TEST PERFORMANCE:\n"
            performance_text += f"  Samples: {test_performance.get('samples_count', 'N/A'):,}\n"
            performance_text += f"  Período: {test_performance.get('test_days', 'N/A')} días\n"
            performance_text += f"  VaR 10%: {test_performance.get('var_10_mean', 0):.6f}\n\n"

        performance_text += f"META-MODEL:\n"
        if 'meta_model_features' in globals() and meta_model_features is not None:
            performance_text += f"  Features: {len(meta_model_features)}\n"
            performance_text += f"  Status: Ready \n"
        else:
            performance_text += f"  Status: Pending\n"

        ax6.text(0.05, 0.95, performance_text, transform=ax6.transAxes,
                fontsize=9, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))

        plt.suptitle('TFT - Análisis Completo de Resultados (CORREGIDO)',
                    fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

        print("Visualizaciones corregidas creadas exitosamente!")

    except Exception as e:
        print(f"Error en visualizaciones corregidas: {e}")
        import traceback
        traceback.print_exc()

if 'predictions' in globals() and predictions is not None:
    print("CREANDO VISUALIZACIONES CORREGIDAS")
    print("=" * 60)

    plot_tft_results_fixed(
        predictions,
        final_performance if 'final_performance' in globals() else None,
        num_samples=30
    )
else:
    print("No hay predicciones disponibles para visualizar")
    print("Asegúrate de haber ejecutado la celda de generación de predicciones primero")

In [ ]:
def create_comprehensive_final_report():

    print("\n" + "="*80)
    print("REPORTE FINAL COMPLETO - TFT PARA META-MODELO")
    print("="*80)

    print("\nDATOS PROCESADOS:")
    if all(var in globals() for var in ['train_raw', 'val_raw', 'test_raw']):
        total_records = len(train_raw) + len(val_raw) + len(test_raw)
        print(f"  Train: {len(train_raw):,} registros")
        print(f"  Validation: {len(val_raw):,} registros")
        print(f"  Test: {len(test_raw):,} registros")
        print(f"  Total procesado: {total_records:,} registros")

        start_date = min(train_raw['timestamp'].min(), val_raw['timestamp'].min(), test_raw['timestamp'].min())
        end_date = max(train_raw['timestamp'].max(), val_raw['timestamp'].max(), test_raw['timestamp'].max())
        total_days = (end_date - start_date).days
        print(f"  Período total: {total_days} días ({start_date.date()} → {end_date.date()})")

    print("\n MODELO TFT:")
    if 'trained_model' in globals():
        total_params = sum(p.numel() for p in trained_model.parameters())
        print(f"  Parámetros: {total_params:,} (~{total_params/1e6:.1f}M)")
        print(f"  Cuantiles: [2%, 10%, 25%, 50%, 75%, 90%, 98%]")
        if 'training_data' in globals():
            print(f"  Encoder length: {training_data.max_encoder_length}")
            print(f"  Prediction length: {training_data.max_prediction_length}")

    print("\n PREDICCIONES:")
    if 'predictions' in globals() and predictions is not None:
        pred_shape = predictions['predictions'].shape
        print(f"  Shape: {pred_shape}")
        print(f"  Samples: {pred_shape[0]:,}")
        print(f"  Horizontes: {pred_shape[1]} (5min-{pred_shape[1]*5}min)")
        print(f"  Cuantiles: {pred_shape[2]}")
        print(f"  Mediana promedio: {predictions['median'].mean():.6f}")
        print(f"  Incertidumbre promedio: {predictions['uncertainty'].mean():.6f}")

    print("\nPERFORMANCE TEST:")
    if 'final_performance' in globals() and final_performance is not None:
        print(f"  Samples test: {final_performance.get('samples_count', 'N/A'):,}")
        print(f"  Días test: {final_performance.get('test_days', 'N/A')}")
        print(f"  Horizonte máximo: {final_performance.get('prediction_horizon_minutes', 'N/A')}")
        print(f"  Mediana test: {final_performance.get('median_mean', 0):.6f} ± {final_performance.get('median_std', 0):.6f}")
        print(f"  VaR 10%: {final_performance.get('var_10_mean', 0):.6f}")
        print(f"  VaR 5%: {final_performance.get('var_5_mean', 0):.6f}")

    print("\nMETA-MODELO FEATURES:")
    if 'meta_model_features' in globals() and meta_model_features is not None:
        print(f" Total features: {len(meta_model_features)}")

        feature_types = {
            'quantiles': [k for k in meta_model_features.keys() if any(q in k for q in ['q02', 'q10', 'q25', 'q75', 'q90', 'q98', 'median'])],
            'uncertainty': [k for k in meta_model_features.keys() if any(u in k for u in ['uncertainty', 'iqr', 'skew'])],
            'directional': [k for k in meta_model_features.keys() if any(d in k for d in ['prob_', 'strong_', 'neutral'])],
            'risk': [k for k in meta_model_features.keys() if any(r in k for r in ['var_', 'cvar_'])],
            'confidence': [k for k in meta_model_features.keys() if 'confidence' in k],
            'volatility': [k for k in meta_model_features.keys() if 'vol' in k],
            'momentum': [k for k in meta_model_features.keys() if 'momentum' in k],
        }

        for category, features in feature_types.items():
            if features:
                print(f"     {category.title()}: {len(features)} features")

        print(f"   Formato: Listo para integración en ensemble")

    print("\n ESTADO DEL PROYECTO:")
    print(f"   Pipeline de datos: COMPLETADO")
    print(f"   Feature engineering: COMPLETADO")
    print(f"   Limpieza robusta: COMPLETADO")
    print(f"   División 3-way: COMPLETADO")
    print(f"   Modelo TFT: COMPLETADO")
    print(f"   Entrenamiento: COMPLETADO")
    print(f"   Predicciones multi-cuantil: COMPLETADO")
    print(f"   Evaluación test: COMPLETADO")
    print(f"   Features meta-modelo: COMPLETADO")

    print("\n🎯 PRÓXIMOS COMPONENTES DEL META-MODELO:")
    next_models = [
        "1. XGBoost_GPU - Clasificación direccional multi-clase",
        "2. LSTM_Attention - Sequence-to-sequence forecasting",
        "3. CNN_1D_LSTM - Patrones de price action",
        "4. FinBERT_Local - Análisis de sentiment",
        "5. TabNet_MultiSource - Features tabulares avanzados",
        "6. VAR_Enhanced - Modelo econométrico",
        "7. Volume_CNN - Microestructura del mercado",
        "8. GCN_Simple - Relaciones cross-asset",
        "9. Neural_Network_Stacking - Meta-learner final"
    ]

    for model in next_models:
        print(f"   {model}")

    print("\nVARIABLES DISPONIBLES PARA CONTINUAR:")
    variables = [
        "trained_model - Modelo TFT entrenado",
        "predictions - Predicciones multi-cuantil validation",
        "test_predictions_np - Predicciones test",
        "meta_model_features - Features para ensemble",
        "final_performance - Métricas de performance",
        "train_raw, val_raw, test_raw - Datos limpios"
    ]

    for var in variables:
        print(f"  {var}")

    print("\n" + "="*80)
    print(" TFT COMPLETAMENTE IMPLEMENTADO Y VALIDADO")
    print(" LISTO PARA INTEGRACIÓN EN META-MODELO ENSEMBLE")
    print(" PRIMER COMPONENTE DEL SISTEMA COMPLETADO")
    print("="*80)

create_comprehensive_final_report()

tft_status = {
    'component': 'TFT (Temporal Fusion Transformer)',
    'status': 'COMPLETADO ',
    'ready_for_ensemble': True,
    'features_generated': len(meta_model_features) if 'meta_model_features' in globals() and meta_model_features is not None else 0,
    'next_component': 'XGBoost_GPU'
}

print(f"\n ESTADO ACTUAL: {tft_status}")